In [1]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.
import kagglehub
samira1992_promoter_or_not_bioinformatics_dataset_path = kagglehub.dataset_download('samira1992/promoter-or-not-bioinformatics-dataset')

print('Data source import complete.')


/home/samuel.assis/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Data source import complete.


In [2]:
# Imports
import torch
import torch.nn as nn
from torchvision import datasets
from torchvision import transforms
import numpy as np
import os
import torch.nn.functional as F
import torch.optim as optim
import sys
import pandas as pd
import re
from torch.utils.data import (TensorDataset, DataLoader, RandomSampler,
                              SequentialSampler)
from sklearn.model_selection import train_test_split
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import clear_output


device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [32]:
def load_data_kaggle():
	promoter_ = pd.read_csv('/kaggle/input/promoter-or-not-bioinformatics-dataset/promoter.csv')
	non_promoter_ = pd.read_csv('/kaggle/input/promoter-or-not-bioinformatics-dataset/non_promoter.csv')
	patterns_table_ = pd.read_csv('/kaggle/input/promoter-or-not-bioinformatics-dataset/bendability.tsv', header=None, names=['codon','score'], sep='\t')
	promoter_ = promoter_.to_numpy()
	non_promoter_ = non_promoter_.to_numpy()

	return promoter_, non_promoter_, patterns_table_

def load_data_local():
	promoter_ = pd.read_csv(os.path.join(samira1992_promoter_or_not_bioinformatics_dataset_path, 'promoter.csv'))
	non_promoter_ = pd.read_csv(os.path.join(samira1992_promoter_or_not_bioinformatics_dataset_path, 'non_promoter.csv'))
	patterns_table_ = pd.read_csv(os.path.join(samira1992_promoter_or_not_bioinformatics_dataset_path, 'bendability.tsv'), header=None, names=['codon','score'], sep='\t')
	promoter_ = promoter_.to_numpy()
	non_promoter_ = non_promoter_.to_numpy()

	return promoter_, non_promoter_, patterns_table_

def tokenize_stride1(sequences, max_length = 250, k=3):
    adjusted_seqs = [seq[:max_length].ljust(max_length, "N").lower() for seq in sequences.flatten()]
    return np.array([[seq[i:i+k] for i in range(len(seq) - k + 1)] for seq in adjusted_seqs])

def one_hot_encoding(sequences_kmerized, codons):

    encoded_codons=[]
    for codon in sequences_kmerized:
        if codon not in codons.keys():
            encoded_codons.append(np.zeros(64))
        else:
            encoded_codons.append(codons[codon])

    return np.array(encoded_codons).T.astype(np.float32)

def data_loader(train_inputs, val_inputs, train_labels, val_labels,
                batch_size=10):
    """
    Convert train and validation sets to torch.Tensors and load them to
    DataLoader.
    """

    # Convert data type to torch.Tensor
    train_inputs, val_inputs, train_labels, val_labels =\
    tuple(torch.tensor(data) for data in
          [train_inputs, val_inputs, train_labels, val_labels])

    # Create DataLoader for training data
    train_data = TensorDataset(train_inputs, train_labels)
    train_sampler = RandomSampler(train_data)
    train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

    # Create DataLoader for validation data
    val_data = TensorDataset(val_inputs, val_labels)
    val_sampler = SequentialSampler(val_data)
    val_dataloader = DataLoader(val_data, sampler=val_sampler, batch_size=batch_size)

    return train_dataloader, val_dataloader

def train(net, dataloader, epochs=1, lr=0.01, momentum=0.9, decay=0.0, verbose=1):
	"""
	Trains a neural network. Returns a 2d numpy array, where every list 
	represents the losses per epoch.
	"""
	net.to(device)
	losses_per_epoch = []
	#criterion = nn.CrossEntropyLoss()
	criterion = nn.BCEWithLogitsLoss()
	optimizer = optim.SGD(net.parameters(), lr=lr, momentum=momentum, weight_decay=decay)
	for epoch in range(epochs):
		sum_loss = 0.0
		losses = []
		for i, batch in enumerate(dataloader, 0):
			# get the inputs; data is a list of [inputs, labels]
			inputs, labels = batch[0].to(device), batch[1].to(device).float()
			print(f"Inputs dtype: {inputs.dtype}, Labels dtype: {labels.dtype}")
			print(f"Inputs length: {len(inputs)}, Labels length: {len(labels)}")
			# zero the parameter gradients
			optimizer.zero_grad()

			# forward + backward + optimize
			outputs = net(inputs)                 
			outputs = outputs.view(-1)
            
			print('loss')
			loss = criterion(outputs, labels)
			loss.backward()
			optimizer.step()

			# print statistics
			losses.append(loss.item())
			sum_loss += loss.item()
			if i % 100 == 99:    # print every 100 mini-batches
				if verbose:
					print('[%d, %5d] loss: %.3f' %
						(epoch + 1, i + 1, sum_loss / 100))
				sum_loss = 0.0
		# print(len(losses))
		losses_per_epoch.append(np.mean(losses))

	return losses_per_epoch

class Net(nn.Module):
    def __init__(self):
        super().__init__()

        self.linear_relu_stack = nn.Sequential(
        nn.Conv1d(in_channels=64, 
                out_channels=128,
                kernel_size=6,
                stride=3,
                padding=1),
        nn.ReLU(),
        nn.MaxPool1d(3, 2, 1),

        nn.Conv1d(in_channels=128, 
                  out_channels=10,
                  kernel_size=6,
                  stride=3,
                  padding=1),
        nn.ReLU(),
        nn.MaxPool1d(3, 2, 1),

        nn.Flatten(),
        nn.Dropout(p=0.5),
        nn.Linear(70, 1),
        nn.Sigmoid())
        

    def forward(self, x):
        #print(f"Input shape: {x.shape}")  # Debugging: Print input shape
        print(f"Input shape: {x.shape}")  # Debugging: Print input shape
        for layer in self.linear_relu_stack:
             print(f"Before {layer.__class__.__name__}: {x.shape}")
             x = layer(x)
             print(f"After {layer.__class__.__name__}: {x.shape}")
        return  x




In [26]:
#promoter, non_promoter, codon_score = load_data_kaggle()

promoter, non_promoter, codon_score = load_data_local()

codon_score_dict = dict(zip(codon_score['codon'], codon_score['score']))

all_raw_sequences = np.concatenate([promoter[0:3000], non_promoter[0:3000]])

all_labels = labels = np.concatenate([
    np.ones(len(promoter[0:3000]), dtype=int),      #1 promoter
    np.zeros(len(non_promoter[0:3000]), dtype=int)  #0 non-promoter
])

codon_to_one_hot = {codon: np.eye(64)[i] for i, codon in enumerate(codon_score_dict)}

sequences_kmerized = tokenize_stride1(all_raw_sequences)

one_hot_seqs = [one_hot_encoding(seq, codons=codon_to_one_hot) for seq in sequences_kmerized]


In [14]:

train_inputs, test_inputs, train_labels, test_labels = train_test_split(
    one_hot_seqs, all_labels, train_size=0.8, test_size=0.2, random_state=42)

train_dataloader, test_dataloader = data_loader(train_inputs, test_inputs, train_labels, test_labels, batch_size=64)



In [33]:
# Creating net and training
os.environ['CUDA_LAUNCH_BLOCKING']="1"
os.environ['TORCH_USE_CUDA_DSA'] = "1"
model = Net()
losses = train(model, train_dataloader, epochs=2, lr=0.01, decay = 0.75)

Inputs dtype: torch.float32, Labels dtype: torch.float32
Inputs length: 64, Labels length: 64
Input shape: torch.Size([64, 64, 248])
Before Conv1d: torch.Size([64, 64, 248])
After Conv1d: torch.Size([64, 128, 82])
Before ReLU: torch.Size([64, 128, 82])
After ReLU: torch.Size([64, 128, 82])
Before MaxPool1d: torch.Size([64, 128, 82])
After MaxPool1d: torch.Size([64, 128, 41])
Before Conv1d: torch.Size([64, 128, 41])
After Conv1d: torch.Size([64, 10, 13])
Before ReLU: torch.Size([64, 10, 13])
After ReLU: torch.Size([64, 10, 13])
Before MaxPool1d: torch.Size([64, 10, 13])
After MaxPool1d: torch.Size([64, 10, 7])
Before Flatten: torch.Size([64, 10, 7])
After Flatten: torch.Size([64, 70])
Before Dropout: torch.Size([64, 70])
After Dropout: torch.Size([64, 70])
Before Linear: torch.Size([64, 70])
After Linear: torch.Size([64, 1])
Before Sigmoid: torch.Size([64, 1])
After Sigmoid: torch.Size([64, 1])
loss
Inputs dtype: torch.float32, Labels dtype: torch.float32
Inputs length: 64, Labels lengt

In [7]:
torch.cuda.is_available() 

True